# Integrating tools/function with LLM using Azure OpenAI

In [1]:
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [13]:
api_key = "xxxxxxxxxxxxx"
api_version = "2023-07-01-preview" # "2023-05-15"
azure_endpoint = "https://xxxxxxxxx.openai.azure.com/"
model_name = "gpt-4o"

In [14]:
from openai import AzureOpenAI
import json
import requests

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version=api_version,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=azure_endpoint,
    api_key = api_key,

)

In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

## Supported models
Not all model versions are trained with function calling data. Function calling is supported with the following models: gpt-4, gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4-0613, gpt-3.5-turbo, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, and gpt-3.5-turbo-0613

In addition, parallel function calls is supported on the following models: gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-3.5-turbo-0125, and gpt-3.5-turbo-1106

In [15]:
# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [16]:
owm_api = "29af1cea50a401d8e624eea4660b3f59"

# create a dummy function to respond temperature
def get_current_weather2(location, unit="fahrenheit"):
  "This function can be used to fetch the current weather information when provided with details such as location and unit for temperature"

  url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={owm_api}"
  response = requests.get(url)
  temp = response.json()['main']['temp']
  forecast = [response.json()['weather'][0]['main'],response.json()['weather'][0]['description']]

  weather_info = {
      "location":location,
      "temperature":temp,
      "unit":'Kelvin',
      "forecast":forecast
  }
  return json.dumps(weather_info)

In [17]:
get_current_weather("Paris")

'{"location": "Paris", "temperature": "22", "unit": "fahrenheit"}'

In [18]:
get_current_weather2("Manila")

'{"location": "Manila", "temperature": 301.42, "unit": "Kelvin", "forecast": ["Clouds", "scattered clouds"]}'

In [19]:
messages = [{"role":"user",'content':"what is AI?"}]
results = client.chat.completions.create(model = model_name, messages = messages)
results.choices[0].message

ChatCompletionMessage(content="Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are designed to think and act like humans. These systems are able to perform tasks that typically require human intelligence, including understanding natural language, recognizing patterns, solving problems, and making decisions. AI can be categorized into various types based on its capabilities:\n\n1. **Narrow AI (Weak AI)**: This type of AI is designed to perform a specific task or a narrow range of tasks. Examples include virtual assistants like Siri or Alexa, recommendation systems on Netflix, and self-driving car technologies. Narrow AI operates under a limited set of constraints and parameters.\n\n2. **General AI (Strong AI)**: This is still theoretical and represents AI systems that possess the ability to understand, learn, and apply knowledge in a way that is indistinguishable from human cognitive abilities. General AI would be capable of performing any in

In [20]:
tools = [{
                            "type":"function",
                            "function":{"name": "get_current_weather",
                           "description": "This function can be used to fetch the current weather information when provided with details such as location and unit for temperature",
                           "parameters": {
                               "type": "object",
                               "properties": {"location": {
                                   "type": "string",
                                   "description": "The name of city/state/country for which weather information is to be fetched",},},
                               "required": ["location",],},},}]



def get_response(messages, tools,model=model_name):
  response = client.chat.completions.create(model = model, messages = messages, tools=tools, tool_choice='auto', temperature=0.5,)
  response = response.choices[0].message
  tool_calls = response.tool_calls

  try:
    if tool_calls:
      print("Making a function call")
      # get available functions
      available_functions = {
          "get_current_weather":get_current_weather,
      }
      print(tool_calls)

      # get details to call the function assuming there is only one function call
      func_name = tool_calls[0].function.name
      func_to_call = available_functions[func_name]
      func_args = json.loads(tool_calls[0].function.arguments)

      # call the external api by calling the function
      func_response = func_to_call(**func_args)

      # again make a call to openai api to communicate results from external function/API
      messages.append(response)
      messages.append(
          {"tool_call_id":tool_calls[0].id,
          "role":"tool",
          "name":func_name,
          "content":str(func_response)}
      )

      second_response = client.chat.completions.create(
      model = model_name,
      messages = messages)
      return second_response
    else:
      return response.content
  except Exception as e:
      print("Error occurred ",e)
      return response




In [21]:
messages = [{"role": "user", "content": "Provide a 2 line explanation for AI"}]
response = get_response(messages, tools)
print(response)

Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It involves various technologies and algorithms that enable computers to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.


In [23]:
messages = [{"role": "user", "content": "How is the weather in Tokyo today?"}]
response = get_response(messages, tools)
print(response.choices[0].message.content)

Making a function call
[ChatCompletionMessageToolCall(id='call_NP0ZCZ1TH7HKXLsFohGJ60HY', function=Function(arguments='{"location":"Tokyo"}', name='get_current_weather'), type='function')]
The current temperature in Tokyo is 10°F. If you need more detailed information such as humidity, wind speed, or weather conditions, please let me know!
